# Visualize Camera

SPDX-License-Identifier: MIT  
SPDX-FileCopyrightText: 2022 Daisuke Sato

https://github.com/Tiryoh/mini-pupper-jupyter-notebooks

## Set Network configuration

In [ ]:
# NETWORK_INTERFACE="eth0"
NETWORK_INTERFACE="wlan0"

import netifaces as ni
ip_address = ni.ifaddresses(NETWORK_INTERFACE)[ni.AF_INET][0]['addr']
print(ip_address)

In [ ]:
import os

# Set self ip address
os.environ['ROS_IP'] = ip_address

# Set ROS_MASTER_URI
ROS_MASTER_IP=ip_address
# ROS_MASTER_IP="192.168.10.14" # custom

os.environ['ROS_MASTER_URI'] = 'http://{}:11311'.format(ROS_MASTER_IP)
print("ROS_MASTER_IP is {}".format(ROS_MASTER_IP))

### Check environment  

It should outputs like this:

```
192.168.10.23
http://192.168.10.14:11311
```

If the result is `http://localhost:11311`, re-run the above procedure.

In [ ]:
# Check Env
!echo $ROS_IP
!echo $ROS_MASTER_URI

## Import libraries

In [ ]:
from sidecar import Sidecar
# Using jupyros(jupyter-ros) 0.5.0 from conda for now
try:
    from jupyros import ros3d
except ImportError:
    from jupyros.ros1 import ros3d

# Check jupyros version
import jupyros
print(jupyros.__version__)

## Subscribe camera image

### Check topic

If it shows `ERROR: Unable to communicate with master!` or `ERROR: Unknown topic /image_raw`, re-run the network configuration step.

In [ ]:
!rostopic info /image_raw

In [ ]:
from IPython.display import display
import ipywidgets.widgets as widgets


image_widget = widgets.Image(format='jpeg', width=300, height=300)
try:
    sc = Sidecar(title='Camera Sidecar Output')
    with sc:
        display(widgets.VBox([image_widget]))
except:
    display(widgets.VBox([image_widget]))

In [ ]:
import cv2
from cv_bridge import CvBridge, CvBridgeError
from io import BytesIO

bridge = CvBridge()
scale = 0.5

def callback(msg):
    try:
        cv_image = bridge.imgmsg_to_cv2(msg, "bgr8")
    except CvBridgeError as e:
        rospy.logerr("CvBridge Error: {0}".format(e))

    height = cv_image.shape[0]
    width = cv_image.shape[1]
    _, frame = cv2.imencode('.jpg', cv2.resize(cv_image, (int(width*scale), int(height*scale))), [int(cv2.IMWRITE_JPEG_QUALITY), 50])
    frame_buffer = BytesIO(frame)
    image_widget.value = frame_buffer.getvalue()

In [ ]:
import jupyros
import rospy
from sensor_msgs.msg import Image
rospy.init_node("image_subscriber")
jupyros.subscribe("/image_raw", Image, callback)